In [1]:
#import dependencies

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder

In [2]:
#Read in csv files, create dataframes

tuition_df = pd.read_csv('Resources\College_Pricing_Trends_76-24.csv')
underemployed_df = pd.read_csv('Resources\\Underemployed.csv')
unemployed_df = pd.read_csv('Resources\\Unemployed.csv')
wages_df = pd.read_csv('Resources\Wages.csv')

In [3]:
# Create function to drop unwanted columns that contain the same string

def drop_colums(df, column_contains):
    df = df.loc[:, ~df.columns.str.contains(column_contains)]
    return df

In [4]:
# Apply function

underemployed_df = drop_colums(underemployed_df, 'Unnamed')
wages_df = drop_colums(wages_df, 'Unnamed')

In [5]:
# Create function to convert date columns to desired 'Year' format

def convert_date(df):    
    df['Date'] = pd.to_datetime(df['Date'])
    df_august = df[df['Date'].dt.month == 8]
    df_august = df_august.reset_index(drop=True)
    df_august['Date'] = df_august['Date'].dt.year
    df_auguest = df_august.rename(columns={'Date':'Year'})
    return df_august


In [6]:
# Run dfs through function

unemployed_df_clean = convert_date(unemployed_df)
underemployed_df_clean = convert_date(underemployed_df)


In [7]:
# Convert 'Date' column in wages_df

wages_df['Date'] = pd.to_datetime(wages_df['Date'])
wages_df['Date'] = wages_df['Date'].dt.year

In [8]:
# Rename 'Date' to 'Year'

unemployed_df_clean = unemployed_df_clean.rename(columns={'Date':'Year'})
underemployed_df_clean = underemployed_df_clean.rename(columns={'Date':'Year'})
wages_df_clean = wages_df.rename(columns={'Date':'Year'})


In [9]:
display(underemployed_df_clean.head())
display(unemployed_df_clean.head())
display(wages_df_clean.head())

,Year,Recent graduates,College graduates
0,1990,43.8,33.5
1,1991,44.4,33.9
2,1992,47.3,34.7
3,1993,46.8,34.4
4,1994,46.5,33.7


,Year,Young workers,All workers,Recent graduates,College graduates
0,1990,7.9,5.3,3.7,2.4
1,1991,9.9,6.6,4.8,3.0
2,1992,11.0,7.4,4.9,3.3
3,1993,10.1,6.6,4.0,2.9
4,1994,9.0,5.8,4.3,2.7


,Year,Bachelor's degree: 25th percentile,Bachelor's degree: median,Bachelor's degree: 75th percentile,High school diploma: median
0,1990,"39,984","52,353","67,188","38,073"
1,1991,"37,393","49,427","64,470","35,673"
2,1992,"37,540","50,054","62,567","33,369"
3,1993,"36,457","48,610","62,788","34,230"
4,1994,"35,387","47,380","61,199","33,561"
